In [ ]:
import sys
import json
import numpy as np
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import matplotlib.image as imgs
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary as summary
from torchvision import transforms
from PIL import Image
import sys
from train import NeuralNetwork
from train import NeuralNetwork2

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import platform
import pandas as pd

#이미지 분할 함수

def create_image():
    
    pictureRemove = os.listdir('./picture')
    for i  in pictureRemove:
        os.remove("./picture/" + i)
    count = 1
    file_list = os.listdir('./test_data')
    for file in file_list:
        src = cv2.imread('./test_data/'+file)

        x,y,z = src.shape

        x_filter = int(x/4) 
        y_filter = int(y/4)
        x_last = x
        y_last = y
        
        try:
            for j in range(1, int(y_last / y_filter)):
                for i in range(1, int(x_last / x_filter)):
                    #roi : y1 y2 x1 x2
                    y1 = (y_filter*j-y_filter)
                    y2 = (y_filter*j)
                    x1 = (x_filter*i-x_filter)
                    x2 = (x_filter*i)
                    roi = src[y_filter*j-y_filter:y_filter*j, x_filter*i-x_filter:x_filter*i]
                    print(y_filter*j-y_filter,y_filter*j, x_filter*i-x_filter, x_filter*i)
                    cv2.imwrite('./picture/{}_id_{}_{}_{}_{}.jpg'.format(count, int((x2+x1)/2),int((y2+y1)/2), int(x2-x1), int(y2-y1)), roi)
            count += 1
        except:
            pass

    

if __name__ == '__main__':

    device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )

    model = NeuralNetwork2().to(device)
    model.load_state_dict(torch.load('./model/best/image_best.pth'))
    model.eval().to(device)
    with open('./one.json') as f:
        json_object = json.load(f)
    
    transforms_train = transforms.Compose([ transforms.Resize((128, 128)),
                                transforms.ToTensor(),
                                # transforms.RandomRotation(10.)
                                ])


    count = 0
    create_image()
    
    file_list = os.listdir('./picture')
    for file in file_list:
        print(file)
        src = Image.open('./picture/'+file).convert("RGB")
        #좌표를 저장하고 split "_" 를 통해 예측 및 데이터 저장
        name = file[:-4].split("_")
        img = transforms_train(src).to(device)
        img  = torch.unsqueeze(img, 0)
        output = model.forward(img).to(device)
        _, predicted = torch.max(output, 1)
        if predicted.item() < 3:
            json_object['annotations'].append({
                "id": int(count),
                "image_id": int(name[0]),
                "category_id" : int(predicted.item()),
                "bbox": [ 
                    int(name[2]),
                    int(name[3]),
                    int(name[4]),
                    int(name[5]),
                    ],
                "area": float(int(name[4])*int(name[5])),
                "iscrowd": 0
            })
        count += 1
    print(json_object['annotations'])
    with open('./result000.json',"w") as f:
        json.dump(json_object, f , indent=4)    
            
        